In [135]:
import re
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match
header_match = re.compile(r"^(?:([\*\#]*) )?([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
frs_match = re.compile(r"([FU]RS\d+)\.?([\d\.]+)?").match
num_escape_match = re.compile(r"^(\*{2})(\#+)").match
image_caption_match = re.compile(r"(Image \d+)").match

class FRSParser():
    VALID_INDENTS = "*+"
    
    # inputs
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    LINE_NUM_ESCAPE = 4
    LINE_IMG_CAPTION = 5
    
    # states
    ST_INIT = 0
    ST_NONUM = 1
    ST_FRS = 2
    
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = []
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
        self.sm_st = self.ST_INIT
        self.sm = [
            # ST_INIT             ST_NONUM             ST_FRS
            [self.set_frs,        self.set_frs,        self.set_frs],        # LINE_FRS
            [None,                None,                self.set_subitem],    # LINE_SUBITEM
            [None,                None,                None],                # LINE_OTHER
            [self.set_header,     self.set_header,     self.set_header],     # LINE_HEADER
            [self.set_num_escape, self.set_num_escape, self.set_num_escape], # LINE_NUM_ESCAPE
            [None,                None,                None],                # LINE_IMG_CAPTION
        ]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
        
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
    
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def feed(self, line):
        ev, arg = self._parse_input(line)
        func = self.sm[ev][self.sm_st]
        if func:
            st, rv = func(arg)
            self.sm_st = st
        else:
            rv = line
        return rv
    
    def _parse_input(self, line): 
        """Parse the input line to determine the 
        type of input, and any arguments to be passed
        to the input event handler.
        """
        m = header_match(line)
        if m:
            ind, fmt, s, fmt2, _ = m.groups()

            # Verify format to avoid false positive
            # if other * or + characters appear in the string
            if fmt == fmt2[::-1]:
                m2 = frs_match(s)
                if m2:
                    typ = self.LINE_FRS
                    frs, nums = m2.groups()
                    arg = (line, ind, fmt, frs, nums)
                    return typ, arg
                if image_caption_match(s):
                    return self.LINE_IMG_CAPTION, ()
                else:
                    typ = self.LINE_HEADER
                    arg = (line, ind, fmt, s)
                return typ, arg
        m = subitem_match(line)
        if m:
            typ = self.LINE_SUBITEM
            arg = line, len(m.group(1) or "")
            return typ, arg
        m = num_escape_match(line)
        if m:
            typ = self.LINE_NUM_ESCAPE
            arg = line,
            return typ, arg
        return self.LINE_OTHER, (line,)
    
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def parse_text(self, text):
        for line in text.splitlines():
            yield self.feed(line)
    
    # State machine methods
                                
    def set_frs(self, arg):
        line, indent, fmt, frs, nums = arg
        self.ilvl = len(indent or "")
        self.fmt = fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        return self.ST_FRS, line
    
    def set_header(self, arg):
        return self.ST_NONUM, arg[0]
    
    def set_subitem(self, arg):
        line, n = arg
        self.set_indent(n)
        fmt = self.frs_string()
        try:
            _, tail = line.split(" ", 1)
        except ValueError:
            v = fmt
        else:
            v = " ".join((fmt, tail))
        return self.sm_st, v
    
    def set_num_escape(self, arg):
        line, = arg
        return self.sm_st, line.lstrip("*")
    
NumState = FRSParser

In [136]:
ntests = 0
passed = 0
errors = []
def assert_ab(a, b, equal=True):
    global ntests, passed, errors
    ntests += 1
    try:
        if equal:
            assert a == b, "%r != %r" % (a, b)
        else:
            assert a != b, "%r == %r" % (a, b)
    except AssertionError as e:
        val = str(e)
        if isinstance(a, str) and isinstance(b, str):
            val += "\n" + str(a.splitlines()) + "\n" + str(b.splitlines())
        errors.append(val)
    else:
        passed += 1

def assert_header_match(s, exp):
    assert_ab(header_match(s).groups(), exp)
    
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("*URS*", (None, "*", "URS", "*", None))
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*", ("*", "*", "FRS1234", "*", None))
assert_header_match("*FRS1234.5.6*", (None, "*", "FRS1234.5.6", "*", None))
assert_header_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234.5.6", "+*", None))
assert_header_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234.5.6.7.8", "+*", None))
assert_header_match("* *FRS1234:*", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*:", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", "*", "Bob"))
assert header_match("**") == None
assert header_match("***") == None

def assert_num_escape(s, exp):
    m = num_escape_match(s)
    r = m.groups() if m else None
    assert_ab(r, exp)
    
assert_num_escape("**# Bob", ("**", "#"))
assert_num_escape("**## Bob", ("**", "##"))
assert_num_escape("**### Bob", ("**", "###"))
assert_num_escape("# Bob", None)
assert_num_escape("## Bob", None)
assert_num_escape("### Bob", None)
assert_num_escape("* Bob", None)
assert_num_escape("** Bob", None)
assert_num_escape("*** Bob", None)

def assert_img_match(s):
    m = image_match(s)
    assert_ab(bool(m), True)
    
assert_img_match("Image 1")

def set_str(st, s):
    ind, fmt, fs, _, _ = header_match(s).groups()
    frs, nums = frs_match(fs).groups()
    st.set_frs((s, ind, fmt, frs, nums))
    

def test_state(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1*", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1*", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    set_str(st, s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    set_str(st, s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    set_str(st, s)
    st.sm_st = st.ST_FRS
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
## Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:* Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
*** Bob
## Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
*** Bob
** *FRS2908.1.1.1:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
#### Biff
*** Bob
# Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
**** *FRS2908.1.1.1.1.1:* Biff
*** Bob
* *FRS2908.1.2:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.0*
# Bob1
## Bob2
*** Fred1
*** Fred2
## Bob3
## Bob4
## Bob5
*** Fred3
#### Bob6
#### Bob7
#### Bob8
*** Fred4
# Bob9
"""

exp = """*FRS2908.0*
* *FRS2908.0.1:* Bob1
** *FRS2908.0.1.1:* Bob2
*** Fred1
*** Fred2
** *FRS2908.0.1.2:* Bob3
** *FRS2908.0.1.3:* Bob4
** *FRS2908.0.1.4:* Bob5
*** Fred3
**** *FRS2908.0.1.4.1.1:* Bob6
**** *FRS2908.0.1.4.1.2:* Bob7
**** *FRS2908.0.1.4.1.3:* Bob8
*** Fred4
* *FRS2908.0.2:* Bob9"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** Fred:
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** Fred:
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** considered +/- 15 degrees
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS123* 
# Bob
*Image 1*: Bob's image
# Bob"""

exp = """*FRS123* 
* *FRS123.1:* Bob
*Image 1*: Bob's image
* *FRS123.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*URS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*URS2908.0* 
* *URS2908.0.1:* Bob
** considered +/- 15 degrees
* *URS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

def finish():
    print("%d/%d tests passed successfully" % (passed, ntests))
    for e in errors:
        print(e)
    if errors:
        raise Exception("Unit testing suite failed")
finish()

96/96 tests passed successfully


In [5]:
lines=r"""
*+FRS1754+*
# Each MFC shall use a calibration factor m and b used to convert raw voltage to actual flowrate
# Calibration of all MFCs shall be accessible through the Shell UI through manual input of calculated m and b values
# Old system variables related to LPM/V conversion shall be removed.
# No input controls or buttons related to calibration shall be present on the MFC Calibration screen
** User is required to perform external calibration and input the values.

*+Implementation+*
# Introduce m (LPM/V) and b (LPM) to the Cal Factors.cfg and 'cal' tab of the RIO Globals, for each MFC:
## CalMFCAir
### m (LPM/V)
### b (LPM)
## CalMFCCO2
### m (LPM/V)
### b (LPM)
## CalMFCO2
### m (LPM/V)
### b (LPM)
## CalMFCN2
### m (LPM/V)
### b (LPM)
# Wherever Voltage is currently converted to LPM by multiplying the Voltage by the MFC's LPM/V System Variable, change it so it multiplies the Voltage by slope, and then adds the intecept
# Wherever requested LPM is currently converted to Voltage by dividing the LPM by the MFC's LPM/V System Variable, change it so it subtracts intercept from LPM, and then divides by slope
# Remove the LPM/V System Variables, and the corresponding Globals from the Gases tab
# The New Calibration Editor is shown below:
!Cal_Sensor_MFC.png!
Numbered Mockup:
!Cal_Sensor_MFC_Numbered.png!
# Only 'Save' and 'Revert' buttons should be included for the MFC calibration menu.  Do not include the following buttons: 'tar1', 'tar2', '1-point', '2-point', 'zero', 'span'.
# Item 1 will be an additional radio button labelled Air, CO2, N2, and O2. When the user clicks on MFC, Item 1 will be shown in place of the 1-point, 2-point cal radio buttons.
## There will always be one item selected.
## When the program is first opened and navigated to MFC, Air will be selected.
"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+FRS1754+*
* *FRS1754.1:* Each MFC shall use a calibration factor m and b used to convert raw voltage to actual flowrate
* *FRS1754.2:* Calibration of all MFCs shall be accessible through the Shell UI through manual input of calculated m and b values
* *FRS1754.3:* Old system variables related to LPM/V conversion shall be removed.
* *FRS1754.4:* No input controls or buttons related to calibration shall be present on the MFC Calibration screen
** User is required to perform external calibration and input the values.

*+Implementation+*
# Introduce m (LPM/V) and b (LPM) to the Cal Factors.cfg and 'cal' tab of the RIO Globals, for each MFC:
## CalMFCAir
### m (LPM/V)
### b (LPM)
## CalMFCCO2
### m (LPM/V)
### b (LPM)
## CalMFCO2
### m (LPM/V)
### b (LPM)
## CalMFCN2
### m (LPM/V)
### b (LPM)
# Wherever Voltage is currently converted to LPM by multiplying the Voltage by the MFC's LPM/V System Variable, change it so it multiplies the Voltage by slope, and then adds the intecept
# Wherever

In [185]:
is_frs_item = re.compile(r"(?:[\*\#]* )?\*FRS([\d\.]+)\:?\*\:?(.*)").match
lines = clipboard.paste()
lines2 = []
for line in lines.splitlines():
    m = is_frs_item(line)
    if m:
        nums, text = m.groups()
        #line = nums.count(".") * "*" + (" *FRS%s:*" % nums) + text
        #print(line)
        line = "#" * nums.count(".") + text
    lines2.append(line)
#print("\n".join(lines2))
clipboard.copy("\n".join(lines2))

In [190]:
template=  r'''lines=r"""%s
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)
'''
s = template % clipboard.paste()
clipboard.copy(s)

In [189]:
lines=r"""

*+James's Notes+*
I've modified the attached VIs (see note 5), to improve the Select Sensors behavior, and user experience.  This won't require the Web UI code to change at all.  Later builds of the Web UI can further optimize the user experience around this feature.

Note: we really shouldn't implement this without fixing issue #2522 first - if fake 'sensor fail' alarms are still happening, this scheme will make things worse.  While this scheme is generally a huge improvement over what we're using in 2.0, the 2.0 scheme is actually better for recovering from fake sensor failures (I don't see that as a true benefit of the 2.0 scheme, but it's better to stick with it until the #2522 bug is resolved)

*+URS+*
URS3194.2.1.3

*+FRS1583+*
* Note: This FRS item recognizes several independent configuration states for each sensor:
** "Present": System configured to have sensor hardware.
** "Enabled": User configured to allow sensor to be used.
** "Preferred": User configured to always use this sensor if both sensors are present and enabled.
** "Active": Whether system "trusts" the reading of this sensor to be accurate, if present and enabled. 
** "In Use": Currently being used for system PV.
# The system shall be able to be configured whether Sensor A is present, Sensor B is present, or both are Present.
# Users shall be able to Enable or Disable specific sensors if present.
# If both sensors are Enabled, a user shall be able to prefer a sensor. 
# pH failure rate alarms shall not be triggered sensors that are Disabled or not Present.
# Sensors shall be tracked as Active depending on whether they have gone in and out of range:
## A user shall be able to change a sensor from "Inactive" to "Active" through the Web UI
## If only one sensor is present, that sensor shall be automatically made Active if it comes back in range
## If both sensors are present, enabled, and one sensor goes out of range, it shall Inactive if it comes back in range
## If both sensors are present, enabled, and both go out of range:
### The first sensor that comes back in range shall be made Active.
### The second sensor shall be Inactive if it comes back in range
## If the non-preferred sensor is Inactive, it shall be made In Use if it becomes Active. 
# Sensor A shall always be Active and In Use if either of the following are true:
## Hardware only supports sensor A
## Hardware supports sensors A and B, and users have configured it to only use A
## Hardware supports sensors A and B, both are enabled, and Sensor A is preferred.
# Sensor B shall always be Active and In Use if either of the following are true:
## Hardware only supports sensor B
## Hardware supports sensors A and B, and users have configured it to only use B
## Hardware supports sensors A and B, both are enabled, and Sensor B is preferred.
# If both sensors are Present, Enabled, and only one is in range, that sensor shall be In Use.
# Web UI configuration handling:
## Clicking a lock icon shall be interpreted as Enabling or Disabling a sensor.
## Clicking a "DNE" icon shall be interpreted as making a sensor Active.
## Changing a slider shall be interpreted as making a sensor Preferred. 
# User events shall be generated at the appropriate times:
## When a lock is closed: "<Sensor> locked onto <Sensor A/B>"
## When a lock is opened: "<Sensor> sensors unlocked"
## When sensor preference changes: "<Sensor Name A/B> set to preferred"
## When sensor becomes active: "Re-activated <Sensor Name>"


*+Implementation+*
# Changes to Enable Disable and Calibrate VI
# Removed the 'Enable/Disable/Do Nothing' function
# This will now just calibrates, and tell you whether the sensor's PV is in valid range or not.
# "Active" is no longer to be an input or output

# Changes to Pick a Sensor VI
# Determining PV rate change
## Hard-wire False to the Alarm Handler for pH Sensor A Failure (rate) under the following conditions:
### Hardware is only for sensor B
### Hardware is for sensors A and B, and user configuration is to only use sensor B
## Hard-wire False to the Alarm Handler for pH Sensor B Failure (rate) under the following conditions:
### Hardware is only for sensor A
### Hardware is for sensors A and B, and user configuration is to only use sensor A
## If Hardware is for sensors A and B, and user configuration is to use sensors A and B, then what's fed into the Alarm Handlers for pH Sensor A and pH Sensor B for 3.0 will be the same as in 2.0
# Determining the following:
### At least 1 sensor is good - if either Sensor 1 is active or Sensor 2 is active, this will be True
### Sensor 1 is active
### sensor 2 is active
### main sensor is in use
### used value - this is the PV reported by the sensor the software has decided to use
### both sensors good yet measuring different things?
# Under the following conditions, define "Sensor 1 is Active" to equal "Sensor 1 in range", set "Sensor 2 is Active" to False, and "Main Sensor is in use" to True:
### Hardware only supports sensor A
### Hardware supports sensors A and B, and users have configured it to only use A
# Under the following conditions, set "Sensor 1 is Active" to False, define "Sensor 2 is Active" to equal "Sensor 2 in range", and set "Main Sensor is in use" to False:
### Hardware only supports sensor B
### Hardware supports sensors A and B, and users have configured it to only use B
# If the machine has hardware to support 2 sensors and the users have configured it to use 2, and only 1 is in range, make that sensor 'active'
# If the machine has hardware to support 2 sensors and the users have configured it to use 2, and either both or neither sensors are in range, and it's the first time calculating which sensors are "Active", it should do it based solely on which sensors are in use.  It should also use whichever sensor the user prefers.
# If the machine has hardware to support 2 sensors and the users have configured it to use 2, and either both or neither sensors are in range, and it's NOT the first time calculating which sensors are "Active", it should set a sensor to be Active if it's both in range and Active.  It should use the same logic as in 2.0 to determine which sensor should be used (the user-preferred one or the other one).
# If the machine has hardware to support 1 sensor, the Pick a sensor.vi "Sensor User Config Output" should be equal to the "Sensor Hardware" input.  Otherwise, it should equal the "Sensor User Config" input.
# Generating the following alarms:
### A Failure (range)
### B Failure (range)
### Dual Sensor Failure
### Sensor Mismatch
# Only feed the relevant booleans to the Alarm Handlers for these alarms if the hardware and user configuration states the sensors are available.  Otherwise hard-code FALSE to them for the alarms that aren't relevant.  For example, if the hardware supports DO sensors A and B but users are only using a DO sensor plugged in to A, don't bother generating alarms for DO B Failure (range), DO Dual Sensor Failure, or DO Sensor Mismatch alarms.


# Changes to Control Module VIs (Temp, pH and DO)
# These have to be modified because the inputs and outputs of the "Enable Disable Calibrate" and "Pick a Sensor" VIs have been modified.
# The MainXSensorEnable and BackupXSensorEnable globals can be removed (where X stands for pH DO and Temp)
# XSensorHardware and XSensorUserConfig globals need to be added (where X stands for pH DO and Temp)
## These 6 globals are to be enums with 3 options:
*** 0 means "only sensor A"
*** 1 means "only sensor B"
*** 2 means "sensors A and B"
# Besides reading from XUserConfig and feeding it into the Pick a sensor.vi, the control modules must also write to the XUserConfig, feeding the Pick a sensor.vi's "Sensor User Config Output" output to the global.


# Changes to Set Sensor States VI
*Note:* The attached example VI does not have 'null' errors or user event text specified anywhere.  Its error handling is also likely less than ideal.  Last, since it is being modified so dramatically, should all the Shared Variables it is reading and writing internally actually be inputs and outputs, so unit testing can take place appropriately?
# Remove the random floating TempPV and AgModeUser things from the VI
# Determine whether the hardware supports 2 sensors, and reject all requests if it doesn't.
# Determine which action should be taken (val1 of 0, 1 or 8 is unfortunately not enough information to go on).  In the future, when the Web UI is updated to take this new scheme into account, the 'val1' it sends will be all that's needed to determine what action to take.
## Close Lock
### val1 = 0, Web sent currently-used sensor, and sensor set is not locked
### val1 = 1
## Open Lock
### val1 = 0, Web sent currently-used sensor, and sensor set is locked
### val1 = 8, and sensor set is locked
## Change preferred sensor
### val1 = 0, Web did not send currently-used sensor
## Re-activate inactive sensor
### val1 = 8, and sensor set is not locked
# Validate and perform the action
## Close Lock
### Validation (all conditions must be met):
#### Sensor set is not locked
#### Web sent used sensor
### Action: Set the Sensor User Config for the sensor set in question to 0 if the web sent Sensor A, and 1 if the web sent Sensor B
### User Event: <Sensor Set> locked onto <Sensor Name> where Sensor Set is the sensor type the Web sent (either DO pH or Temp), and Sensor Name is the individual sensor the Web sent (DO A, DO B, pH A, etc.)
## Open Lock
### Validation: Sensor set is locked
### Action: Set the Sensor User config for the sensor set in question to 2
### User Event: <Sensor Set> sensors unlocked
## Change preferred sensor
### Validation (all conditions must be met):
#### Sensor set is not locked
#### Web did not send used sensor
### Action: Set 'Sensor A is Primary User' for the sensor set in question to TRUE if the Web sent sensor A, and FALSE if the Web sent sensor B
### User Event: <Sensor Name> set to preferred sensor
## Re-activate inactive sensor
### Validation (all conditions must be met):
#### Sensor set is not locked
#### Sensor is not active
### Actions:
#### Set Sensor User Config to equal the in-use sensor (if A is used, set it to 0; if B is used, set it to 1)
#### Check that the change went through
#### If the change went through, set the Sensor User Config back to 2
#### Confirm that the sensor in question became active
### User Event: re-activated <Sensor Name>


# Changes to Webservice Command Handler
# getSensorStates
## See attached files getSensorStates_Orig and getSensorStates_Mods for exact differences
## Basic difference:
### In 2.0, each controller's "locked" parameter is set to 1 if either sensor in the pair is 'Disabled', otherwise it's unlocked (value 0)
### In 3.0, each controller's "locked" parameter will be set to 1 if the sensor pair's user configuration is less than 2 (0, if they've locked it on A, and 1 if they've locked it on B).  Unlocked would mean either A or B sensors could be used.
# setSensorState
## Just supports mods to Set Sensor States VI
# getStatic
## Should return the SensorHardware parameters for DO pH and Temp as booleans, where the value returned is 0 if the sensor's hardware is either A or B, and 1 if the value is A & B.

*+Action+*
# The new globals are called DOHardware, DOUserConfig, pHHardware, pHUserConfig, TempHardware, TempUserConfig.
# The hardware info for DO, pH, and Temp will be stored with Bioreactor Model information in a file called Bioreactor Configuration.txt.

*+Notes:+*
# I've decided not to force the user-set parameter to be "locked" if the hardware only supports 1 sensor.  For 3.0, all that will accomplish is giving a nice UI to a user who managed to change the globals via recipe (because I plan on making the Globals 1 2 and 3 defaults' user-set parameters match the hardware parameters).  And once the Web UI is updated to take advantage of this new scheme, it will not show sensor pairs if the hardware doesn't support 2 sensors.
# As of now, implementing this would give us the following improvements:
## After setting the "SensorInputs" parameters for Temp, DO and pH at the factory, we wouldn't have to worry about users messing up any other configuration.
## In fact, users wouldn't have to access the Service UI to set which configuration of sensors they're using - the "SensorConfig" parameters for Temp, DO and pH would be set in the Web UI's Select Sensors menu
## If the user has 2 sensors configured, and 1 is inactive but in range, and the 2nd goes out of range, control switches to the 1st sensor, instead of entering "broken sensor" mode.  This is for a situation where the user hasn't yet been able to respond to the 1st sensor failure.
## Users are prevented from changing the preferred sensor or re-enabling a sensor when the lock is closed
## When a user clicks the slider to select an inactive sensor (i.e. it has a DNE on it), the backend software will not only change the user-preferred sensor but also re-activate that sensor.
## If this information is sent in the getVersion call, future versions of the Web UI can use it to determine which sensors to show in the Select Sensors menu and the Calibration menu (currently, that decision is based on the Model).
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)




*+James's Notes+*
I've modified the attached VIs (see note 5), to improve the Select Sensors behavior, and user experience.  This won't require the Web UI code to change at all.  Later builds of the Web UI can further optimize the user experience around this feature.

Note: we really shouldn't implement this without fixing issue #2522 first - if fake 'sensor fail' alarms are still happening, this scheme will make things worse.  While this scheme is generally a huge improvement over what we're using in 2.0, the 2.0 scheme is actually better for recovering from fake sensor failures (I don't see that as a true benefit of the 2.0 scheme, but it's better to stick with it until the #2522 bug is resolved)

*+URS+*
URS3194.2.1.3

*+FRS1583+*
* Note: This FRS item recognizes several independent configuration states for each sensor:
** "Present": System configured to have sensor hardware.
** "Enabled": User configured to allow sensor to be used.
** "Preferred": User configured to always use th

In [191]:
lines=r"""*+Current Behavior:+*
# In the following scenario, user1 will hijack user2's permission and "account" in the sense that whatever user1 does will be logged as user2's actions in the database.
## User1 logs in.
## User2 logs in.
## User2 logs out.
## User1 now inherits user2's name and Config setting privileges. User1's action is logged as user2 in the database.
# Login, logout, and user actions record the wrong usernames.

*+FRS2367+*
# All actions taken by users are checked for the correct user's permissions, then recorded as the correct user.  Even if more than 1 user is logged in at once.  Even if more than 1 user logs in at once, and then 1 or more log out.
# Users cannot hijack other users' sessions
# When a user logs in, all actions done by the user is attributed to that user and not another user or unknown user.

*+Implementation+*
Possible Solution:
Remove globals for sessions, session attributes, and local session with a shift register variable in webservice command handler.vi The local session will include items in the existing local session with session cookies ID and all the permissions for that user.

Webservice Command Handler should be modified to be more modular.

*+Notes:+*
Session cookie ID is randomly generated as far as I can tell. They do not increase serially.
The session currently does not end or get deleted because our code only adds session and does not delete them. However, once it exceeds 50 sessions, it will replace the oldest session with the newest one.
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+Current Behavior:+*
# In the following scenario, user1 will hijack user2's permission and "account" in the sense that whatever user1 does will be logged as user2's actions in the database.
## User1 logs in.
## User2 logs in.
## User2 logs out.
## User1 now inherits user2's name and Config setting privileges. User1's action is logged as user2 in the database.
# Login, logout, and user actions record the wrong usernames.

*+FRS2367+*
* *FRS2367.1:* All actions taken by users are checked for the correct user's permissions, then recorded as the correct user.  Even if more than 1 user is logged in at once.  Even if more than 1 user logs in at once, and then 1 or more log out.
* *FRS2367.2:* Users cannot hijack other users' sessions
* *FRS2367.3:* When a user logs in, all actions done by the user is attributed to that user and not another user or unknown user.

*+Implementation+*
Possible Solution:
Remove globals for sessions, session attributes, and local session with a shift register var